In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git


import sklearn
import sklearn.feature_extraction.text

#import scipy.cluster.hierarchy
import gensim#For topic modeling
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

In [2]:
df = pandas.read_csv('t\df_00.csv', index_col=0)

In [3]:
#Apply our functions
df['tokenized_text'] = df['AB'].apply(lambda x: lucem_illud_2020.word_tokenize(x))
df['normalized_tokens'] = df['tokenized_text'].apply(lambda x: lucem_illud_2020.normalizeTokens(x))

In [4]:
ngTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5, 
                                                                 max_features=1000, min_df=3, 
                                                                 stop_words='english', 
                                                                 norm='l2')
groupsTFVects = ngTFVectorizer.fit_transform(df['AB'])

In [5]:
def dropMissing(wordLst, vocab):
    return [w for w in wordLst if w in vocab]

df['reduced_tokens'] = df['normalized_tokens'].apply(lambda x: dropMissing(x, ngTFVectorizer.vocabulary_.keys()))

In [6]:
dictionary = gensim.corpora.Dictionary(df['reduced_tokens'])

In [7]:
## create a list of tuples containing each token and its count 
## use the first half and save the remainder for testing 
corpus = [dictionary.doc2bow(text) for text in df['reduced_tokens']]

In [8]:
# serialize the corpus as a file and load it 
gensim.corpora.MmCorpus.serialize('abstract.mm', corpus)
abmm = gensim.corpora.MmCorpus('abstract.mm')

In [9]:
# correctly formatted corpus that can be used for topic modeling and induction 
senlda = gensim.models.ldamodel.LdaModel(corpus=abmm, id2word=dictionary, num_topics=20, alpha='auto', eta='auto')

In [10]:
# sen1Bow = dictionary.doc2bow(df['reduced_tokens'][0])
# sen1lda = senlda[sen1Bow]
# print("The topics of the text: {}".format(df['TI'][0]))
# print("are: {}".format(sen1lda))

In [11]:
ldaDF = pandas.DataFrame({
        'titles' : df['TI'],
        'topics' : [senlda[dictionary.doc2bow(l)] for l in df['reduced_tokens']]
    })

In [12]:
topicsProbDict = {i : [0] * len(ldaDF) for i in range(senlda.num_topics)}

for index, topicTuples in enumerate(ldaDF['topics']):
    for topicNum, prob in topicTuples:
        topicsProbDict[topicNum][index] = prob

for topicNum in range(senlda.num_topics):
    ldaDF['topic_{}'.format(topicNum)] = topicsProbDict[topicNum]

In [13]:
K = senlda.num_topics  # N documents, K topics
topic_labels = ['Topic #{}'.format(k) for k in range(K)]

topicsDict = {}
for topicNum in range(senlda.num_topics):
    topicWords = [w for w, p in senlda.show_topic(topicNum)]
    topicsDict['Topic_{}'.format(topicNum)] = topicWords

wordRanksDF = pandas.DataFrame(topicsDict)
wordRanksDF

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Topic_10,Topic_11,Topic_12,Topic_13,Topic_14,Topic_15,Topic_16,Topic_17,Topic_18,Topic_19
0,care,child,determinant,patient,determinant,use,determinant,factor,factor,factor,determinant,level,rate,care,behavior,difference,factor,patient,factor,child
1,determinant,age,factor,determinant,high,determinant,care,determinant,determinant,status,result,determinant,determinant,age,determinant,child,child,factor,patient,determinant
2,policy,determinant,disease,high,status,patient,group,behavior,risk,incidence,individual,factor,capital,level,change,determinant,year,age,determinant,country
3,public,mental,develop,community,year,care,work,result,patient,determinant,rate,result,population,determinant,need,care,age,determinant,qol,disease
4,service,factor,model,support,result,risk,use,group,population,increase,use,population,age,factor,model,work,result,result,self,life
5,child,year,problem,factor,hiv,self,result,work,self,age,age,measure,result,result,factor,result,determinant,life,result,report
6,age,level,research,ci,age,factor,status,model,analysis,area,disease,include,individual,patient,variable,analysis,risk,risk,age,use
7,research,group,development,disease,child,physical,self,variable,include,high,level,old,public,child,group,self,low,hospital,analysis,result
8,include,result,public,child,bmi,life,report,self,result,self,high,variable,report,activity,intervention,model,high,analysis,support,population
9,level,care,approach,use,care,outcome,community,age,sexual,child,increase,individual,country,method,include,age,life,high,child,level


In [14]:
import pyLDAvis.gensim
lda_model = senlda
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2     -0.040637  0.006022       1        1  9.071821
18     0.009225 -0.000328       2        1  9.024594
1      0.022954  0.028240       3        1  8.548901
5     -0.004066 -0.017389       4        1  6.638688
13     0.013653  0.019070       5        1  6.354971
4      0.040300  0.001977       6        1  6.160116
11     0.002990 -0.001399       7        1  5.730424
16     0.036863  0.015725       8        1  5.402236
17     0.027037 -0.007711       9        1  5.390190
6      0.004561 -0.016717      10        1  4.781031
10     0.007280 -0.029809      11        1  4.167848
12    -0.015995  0.008727      12        1  3.634936
9      0.001532 -0.004450      13        1  3.608120
8     -0.017485 -0.023012      14        1  3.597417
14    -0.074005 -0.039946      15        1  3.426051
3      0.017140  0.001553      16        1  3.279648
19    -0.000541  0.007213      17        1  3.252156
0     -0.050634  0.077542      18        1  3.140615
7      0.007730 -0.025786      19        1  2.402048
15     0.012097  0.000479      20        1  2.388180, topic_info=    Category        Freq        Term       Total  loglift  logprob
213  Default  639.000000        care  639.000000  30.0000  30.0000
149  Default  688.000000     patient  688.000000  29.0000  29.0000
59   Default  649.000000       child  649.000000  28.0000  28.0000
379  Default  305.000000    behavior  305.000000  27.0000  27.0000
191  Default  912.000000      factor  912.000000  26.0000  26.0000
..       ...         ...         ...         ...      ...      ...
142  Topic20   12.299677       level  589.649902  -0.1353  -4.9719
150  Topic20   11.015814  population  478.454010  -0.0366  -5.0821
58   Topic20   10.035642        year  559.427063  -0.2861  -5.1753
89   Topic20    9.471298    increase  413.988403  -0.0429  -5.2332
67   Topic20    9.315108     include  445.598297  -0.1331  -5.2498

[1272 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
538       1  0.135014  absence
538       2  0.108011  absence
538       3  0.054006  absence
538       4  0.081008  absence
538       5  0.027003  absence
...     ...       ...      ...
434      15  0.018283    youth
434      16  0.073131    youth
434      17  0.036566    youth
434      18  0.036566    youth
434      19  0.036566    youth

[7990 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 19, 2, 6, 14, 5, 12, 17, 18, 7, 11, 13, 10, 9, 15, 4, 20, 1, 8, 16])